# Import potrebných knižníc

In [5]:
import shutil
from PIL import Image, ImageDraw
import glob
import os

# Presunutie obrázkov

Pred orezaním obrázkov sme ich presunuli do jedného spoločného súboru za celý rok

In [13]:
#vytvorenie všetkých potrebných priečinkov
input_dir = 'colombia2021'
output_dir = '2021_all'
os.makedirs(output_dir, exist_ok=True)
os.makedirs("2021_crop/2021_all", exist_ok=True)

#kopírovanie všetkých súborov .gif do jedného priečinku pre celý rok
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith('.gif'):
            input_path = os.path.join(root, file)
            output_path = os.path.join(output_dir, file)

            shutil.copy(input_path, output_path)

# Orezanie snímok BU

Prvotné odstránenie okrajov snímky ktoré obsahovali údaje o čase a mieste zhotovenia snímky.

Takto upravené snímky boli následne určené pre ručnú klasifikáciu.

In [ ]:
#zoznam všetkých súborov .gif
files = glob.glob(r'2021_all/*.gif')

#iterácia každého súboru
for filename in files:
    try:
        #otvorenie obrázka a získanie veľkosti
        img = Image.open(filename)
        width, height = img.size
        x = (width - height)//2
        #orezanie do tvaru štvorca
        img_cropped = img.crop((x, 0, x+height, height))
        
        #vytvorenie masky v tvare kruhu
        mask = Image.new('L', img_cropped.size)
        mask_draw = ImageDraw.Draw(mask)
        width, height = img_cropped.size
        mask_draw.ellipse((15, 30, width-35, height-35), fill=255)
        #aplikovanie kruhovej masky
        img_cropped.putalpha(mask)
        img_cropped = img_cropped.convert('RGBA')   
        #uloženie do nového priečinku
        img_cropped.save("2021_crop/" + os.path.splitext(filename)[0] + ".png")
    #výnimka v prípade poškodeného súboru
    except IOError as e:
        print(f"Preskočenie {filename} kvôli {str(e)}")
        continue

Sekundárne orezanie snímkov bolo uskutočnené pre trénovanie klasifikačného modelu ako aj ďalšiu prácu

In [ ]:
#vytvorenie všetkých potrebných priečinkov
input_folder = "2021_crop/2021_all"
output_folder = "2021_small/"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
else:
    for filename in os.listdir(output_folder):
        file_path = os.path.join(output_folder, filename)
        if os.path.isfile(file_path):
            os.unlink(file_path)

In [ ]:
#definovanie menšieho kruhu
smaller_radius = 150
#všetky png obrázky
image_files = [f for f in os.listdir(input_folder) if f.endswith(".png")]

#iterácia pre každý obrázok 
for filename in image_files:
    input_image = Image.open(os.path.join(input_folder, filename))
    
    #vytvorenie novej kruhovej masky
    mask = Image.new("L", input_image.size, 0)
    draw = ImageDraw.Draw(mask)
    width, height = input_image.size
    center = (width // 2, height // 2)
    draw.ellipse((center[0] - smaller_radius, center[1] - smaller_radius, center[0] + smaller_radius, center[1] + smaller_radius), fill=255)
    
    #aplikovanie kruhovej masky
    output_image = Image.new("RGBA", input_image.size)
    output_image.paste(input_image, mask=mask)

    output_filename = os.path.join(output_folder, filename)
    output_image.save(output_filename)